In [2]:
import pandas as pd
# 读取数据
# data = pd.read_excel('./data/info_table（训练+验证集）.xlsx')
data = pd.read_excel('./data/fill_NaN_data.xlsx')

In [46]:
# 转换为统一的时间格式
data.loc[:, '开点'] = pd.to_datetime(data.loc[:, '开点'], format='%H:%M').dt.strftime('%H:%M')
data.loc[:, '到点'] = pd.to_datetime(data.loc[:, '到点'], format='%H:%M').dt.strftime('%H:%M')
data

,序号,车次,始发站,终到站,到点,开点,候车厅,检票口,站台
0,1,K4547/6,成都西,佳木斯,23:40,00:12,综合候乘中心，高架候车区西区,1B,2
1,2,Z362,乌鲁木齐,南通,00:10,00:21,综合候乘中心，高架候车区西区,5B,5
2,3,T308,乌鲁木齐,南昌,00:16,00:28,综合候乘中心，高架候车区西区,1A,5
3,4,D218,兰州,上海,00:42,00:53,综合候乘中心，高架候车区西区,2A,1
4,5,K178,西宁,郑州,00:48,01:02,综合候乘中心，高架候车区西区,8B,2
...,...,...,...,...,...,...,...,...,...
214,215,0G665,西安,西安动车段,23:26,23:15,高架候车区东区,1B,1
215,216,Z294,乌鲁木齐,武昌,23:22,23:40,高架候车区东区,5B,2
216,217,Z128,兰州,厦门北,23:28,23:50,高架候车区东区,1A,5
217,218,K227,广州,兰州,23:41,23:50,高架候车区东区,2A,2


In [3]:
import random
random_data = data.sample(n=random.randint(1,10), random_state=42)
train_info = ""
for i, row in random_data.iterrows():
    train_info += f"车次：{row['车次']}，候车区：{row['候车厅']}，检票口：{row['检票口']}，站台：{row['站台']},始发站：{row['始发站']}，终点站：{row['终到站']}，到点：{row['到点']}，开点：{row['开点']}\n"
print(train_info)

车次：T248，候车区：综合候乘中心，高架候车区东区，检票口：1B，站台：1,始发站：西安，终点站：武昌，到点：18:47，开点：18:36
车次：C243/2，候车区：综合候乘中心，高架候车区东区，检票口：9B，站台：1,始发站：韩城，终点站：安康，到点：10:31，开点：10:51
车次：Z128，候车区：高架候车区东区，检票口：1A，站台：5,始发站：兰州，终点站：厦门北，到点：23:28，开点：23:50
车次：K227，候车区：高架候车区东区，检票口：2A，站台：2,始发站：广州，终点站：兰州，到点：23:41，开点：23:50
车次：K773/2，候车区：高架候车区西区，检票口：13B，站台：5,始发站：杭州，终点站：呼和浩特，到点：04:47，开点：05:40



In [31]:
# 填充缺失的时间

def fill_NaN(data :pd.DataFrame) -> pd.DataFrame:  
    '''
    填充NaN值

    参数:
        data: pd.DataFrame, 数据框
    返回:
        pd.DataFrame, 填充后的数据框
    '''

    # 观察数据集，得知到点和开点之间相差的时间再10-20分钟之内
    # 读取data数据，判断到点，开点时间如果有一个为空时，随机加上或减少一个随机数（10～20）分钟，填充NaN值
    # 还要考虑到跨天的时间转换，其实只有第一条和最后一条出现了跨天，但是第一条没有缺失的时间，只考虑最后一条（完全可以加个少于10分钟的特例，让它不要跨天）
    import random
    from datetime import datetime, timedelta

    # 先备份data
    result = data.copy()

    # 先转化成可以计算的时间格式
    result['开点'] = pd.to_datetime(result['开点'])
    result['到点'] = pd.to_datetime(result['到点'])

    # 先判断是否有跨天的情况
    for i in range(len(result)):
        if result.loc[i, '开点'] > result.loc[i, '到点']:
            result.loc[i, '到点'] += timedelta(days=1)

    # 定义随机时间
    random_minutes = int(random.randint(10, 20))

    # 填充NaN值
     # 处理缺失的到点
    for i in range(len(result)):
        if pd.isna(result.loc[i, '到点']) and not pd.isna(result.loc[i, '开点']):
            result.loc[i, '到点'] = result.loc[i, '开点'] + pd.Timedelta(minutes=random_minutes)
        
        # 处理缺失的开点
        elif pd.isna(result.loc[i, '开点']) and not pd.isna(result.loc[i, '到点']):
            result.loc[i, '开点'] = result.loc[i, '到点'] - pd.Timedelta(minutes=random_minutes)

     # 转回 HH:MM 格式
    for col in ['开点', '到点']:
        result[col] = result[col].dt.strftime('%H:%M')
    
    return result

fill_NaN_data = fill_NaN(data)
fill_NaN_data

/var/folders/2m/tlyy0yg13gn1whrn0f_kzn540000gn/T/ipykernel_84961/3955663180.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result['开点'] = pd.to_datetime(result['开点'])
/var/folders/2m/tlyy0yg13gn1whrn0f_kzn540000gn/T/ipykernel_84961/3955663180.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result['到点'] = pd.to_datetime(result['到点'])


,序号,车次,始发站,终到站,到点,开点,候车厅,检票口,站台
0,1,K4547/6,成都西,佳木斯,23:40,00:12,综合候乘中心，高架候车区西区,1B,2
1,2,Z362,乌鲁木齐,南通,00:10,00:21,综合候乘中心，高架候车区西区,5B,5
2,3,T308,乌鲁木齐,南昌,00:16,00:28,综合候乘中心，高架候车区西区,1A,5
3,4,D218,兰州,上海,00:42,00:53,综合候乘中心，高架候车区西区,2A,1
4,5,K178,西宁,郑州,00:48,01:02,综合候乘中心，高架候车区西区,8B,2
...,...,...,...,...,...,...,...,...,...
214,215,0G665,西安,西安动车段,23:26,23:15,高架候车区东区,1B,1
215,216,Z294,乌鲁木齐,武昌,23:22,23:40,高架候车区东区,5B,2
216,217,Z128,兰州,厦门北,23:28,23:50,高架候车区东区,1A,5
217,218,K227,广州,兰州,23:41,23:50,高架候车区东区,2A,2


In [32]:
# 创建新的.xlsx文件
fill_NaN_data.to_excel('./data/fill_NaN_data.xlsx', index=False)